In [46]:
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc, callback, Output, Input
import dash

In [47]:
df = pl.read_csv("C:/Users/riain/VSCODE_PROJECTS/Data_In_Motion/Projects_Challenges/Data Analysis Challenges/week12/datasets/ds_salaries.csv")
df.head()       

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
i64,i64,str,str,str,i64,str,i64,str,i64,str,str
0,2020,"""MI""","""FT""","""Data Scientist…",70000,"""EUR""",79833,"""DE""",0,"""DE""","""L"""
1,2020,"""SE""","""FT""","""Machine Learni…",260000,"""USD""",260000,"""JP""",0,"""JP""","""S"""
2,2020,"""SE""","""FT""","""Big Data Engin…",85000,"""GBP""",109024,"""GB""",50,"""GB""","""M"""
3,2020,"""MI""","""FT""","""Product Data A…",20000,"""USD""",20000,"""HN""",0,"""HN""","""S"""
4,2020,"""SE""","""FT""","""Machine Learni…",150000,"""USD""",150000,"""US""",50,"""US""","""L"""


In [48]:
# Which role has the highest salary employment wise?
role_highest_salary = df.sort(by="salary_in_usd", descending=True).head(1)
role_highest_salary # the role with highest salary is Principal Data Engineer in the United States

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
i64,i64,str,str,str,i64,str,i64,str,i64,str,str
252,2021,"""EX""","""FT""","""Principal Data…",600000,"""USD""",600000,"""US""",100,"""US""","""L"""


In [49]:
# Which employment types do employers prefer to hire?
prefered_employment_types = df["employment_type"].value_counts()
prefered_employment_types # full time employees

employment_type,counts
str,u32
"""FT""",588
"""CT""",5
"""PT""",10
"""FL""",4


In [50]:
prefered_employment_types_fig = px.bar(data_frame=prefered_employment_types.to_pandas(), x="employment_type", y="counts", template="seaborn",)
prefered_employment_types_fig.update_layout(bargap=0.7, width=600)
prefered_employment_types_fig.show()

In [51]:
# Which role are entry leveled generally hired for?
entry_leveled = df.filter(pl.col("experience_level") == "EN")["job_title"].value_counts().sort(by="counts", descending=True)
entry_leveled # data scientist is the role entry leveled employees are generally hired for

job_title,counts
str,u32
"""Data Scientist…",22
"""Data Analyst""",12
"""Data Engineer""",12
"""Machine Learni…",9
"""Data Science C…",5
"""AI Scientist""",4
"""Research Scien…",4
"""Big Data Engin…",3
"""Computer Visio…",3


In [52]:
entry_leveled_fig = px.bar(entry_leveled.to_pandas(), x="job_title", y="counts", template="seaborn",)
entry_leveled_fig.update_layout(bargap=0.3, height=500)
entry_leveled_fig.update_xaxes(tickangle=-45)
entry_leveled_fig.show()

In [53]:
# Which countries pay the highest for which roles?
country_highest_pay = df.groupby(by=["job_title", "company_location"]).agg([pl.col("salary_in_usd").max()]).sort("salary_in_usd", descending=True)
country_highest_pay # usa pays the highest salary until the 14th row which is a Machine Learning Scientist from Japan that makes $260000

job_title,company_location,salary_in_usd
str,str,i64
"""Principal Data…","""US""",600000
"""Financial Data…","""US""",450000
"""Research Scien…","""US""",450000
"""Applied Machin…","""US""",423000
"""Principal Data…","""US""",416000
"""Data Scientist…","""US""",412000
"""Data Analytics…","""US""",405000
"""Applied Data S…","""US""",380000
"""Director of Da…","""US""",325000


In [54]:
country_highest_pay_fig = px.scatter(data_frame=country_highest_pay.to_pandas(), x="job_title", y="salary_in_usd")
country_highest_pay_fig.update_layout(height=700)
country_highest_pay_fig.show()

In [105]:
# What insights can you find regarding employee demographics?
# I can get the employee residence, if they are working remote, hybrid, or on-site, their experience level, and their salary
emp_residence = df["employee_residence"].value_counts().sort(by="counts", descending=True)
emp_residence_fig = px.bar(data_frame=emp_residence.to_pandas(), x="employee_residence", y="counts", title="Residence Count", template="seaborn") # most of this data is from US residents
emp_residence_fig.show()

In [101]:
remote_choice = df["remote_ratio"].value_counts()
remote_choice = remote_choice.select(pl.col("counts"), pl.col("remote_ratio").apply(lambda x: str(x)).cast(pl.Categorical))
remote_choice

counts,remote_ratio
u32,cat
99,"""50"""
127,"""0"""
381,"""100"""


In [106]:
remote_choice_fig = px.bar(data_frame=remote_choice.to_pandas(), x="remote_ratio", y="counts", template="seaborn")
remote_choice_fig.update_layout(bargap=0.7)
remote_choice_fig.update_xaxes(categoryarray=["0", "50", "100"]) # 0 is onsite 50 is hybrid 100 is remote
remote_choice_fig.show()

In [126]:
# make a graph where it compares the residency of the employee and their remote choice
remote_choice_residence = df.groupby(by=["employee_residence", "remote_ratio"]).agg([pl.col("salary_in_usd").mean()]).sort("salary_in_usd", descending=True)
remote_choice_residence_fig = px.bar(data_frame=remote_choice_residence.to_pandas(), x="employee_residence", y="salary_in_usd", color="remote_ratio", template="ggplot2")
remote_choice_residence_fig.update_layout(bargap=0.7)
remote_choice_residence_fig.show()

In [125]:
# make a graph that compares the employee experience level and their remote choice
remote_choice_experience = df.groupby(by=["experience_level", "remote_ratio"]).agg([pl.col("salary_in_usd").mean()]).sort("salary_in_usd", descending=True)
remote_choice_experience_fig = px.bar(data_frame=remote_choice_experience.to_pandas(), x="experience_level", y="salary_in_usd", color="remote_ratio", template="ggplot2")
remote_choice_experience_fig.update_layout(bargap=0.7)
remote_choice_experience_fig.show()

In [124]:
# make a graph that compares employee residency and company location
residence_company_location = df.groupby(by=["employee_residence", "company_location"]).agg([pl.col("salary_in_usd").mean()]).sort("salary_in_usd", descending=True)
residence_company_location_fig = px.bar(data_frame=residence_company_location.to_pandas(), x="employee_residence", y="salary_in_usd", color="company_location", template="ggplot2")
residence_company_location_fig.update_layout(bargap=0.7)
residence_company_location_fig.show()

In [56]:
# Which experience level has the highest hiring?
exp_lvl_highest_hiring = df["experience_level"].value_counts(sort=True)
exp_lvl_highest_hiring # looks like the most being hired are senior and mid level employees

experience_level,counts
str,u32
"""SE""",280
"""MI""",213
"""EN""",88
"""EX""",26


In [57]:
exp_lvl_highest_hiring_fig = px.bar(data_frame=exp_lvl_highest_hiring.to_pandas(), x="experience_level", y="counts", title="Experience Levels Hired Count", template="seaborn")
exp_lvl_highest_hiring_fig.update_layout(bargap=0.7)
exp_lvl_highest_hiring_fig.show()

In [58]:
# Does company size affect the rate of hiring and pay scale?
size_year = df.groupby(["company_size", "work_year"]).count().sort(by="work_year", descending=False)
size_year

company_size,work_year,count
str,i64,u32
"""L""",2020,33
"""S""",2020,25
"""M""",2020,14
"""L""",2021,119
"""S""",2021,45
"""M""",2021,53
"""M""",2022,259
"""L""",2022,46
"""S""",2022,13


In [59]:
fig = px.bar(data_frame=size_year.to_pandas(), x="company_size", y="count", facet_col="work_year", template="seaborn", title="Company Size Count differed by Work Year")
# fig.update_layout(bargap=0.4)
fig.update_xaxes(categoryarray=["S", "M", "L"])
fig.show() 
# Jobs have increased from 2020 to 2022. Medium jobs seem to have held a record of most jobs in 2022. 

In [60]:
size_salary = df.groupby(["work_year", "company_size"]).agg(pl.col("salary_in_usd").mean().suffix("_mean")).sort(by="salary_in_usd_mean", descending=False)
size_salary.head()

work_year,company_size,salary_in_usd_mean
i64,str,f64
2020,"""S""",70958.56
2021,"""M""",74154.792453
2022,"""S""",77046.538462
2021,"""S""",81509.844444
2020,"""L""",106303.848485


In [61]:
fig2 = px.bar(data_frame=size_salary.to_pandas(), x="company_size", y="salary_in_usd_mean", facet_col="work_year", template="seaborn")
fig2.update_layout(title="Average Salary vs Company Sizes between the years 2020-2022")
fig2.update_xaxes(categoryarray=["S", "M", "L"])
fig2.show()

In [62]:
size_salary_final = df.groupby(["company_size"]).agg(pl.col("salary_in_usd").mean().suffix("_mean")).sort(by="salary_in_usd_mean", descending=False)
size_salary_final

company_size,salary_in_usd_mean
str,f64
"""S""",77632.674699
"""M""",116905.466258
"""L""",119242.994949


In [63]:
fig3 = px.bar(data_frame=size_salary_final.to_pandas(), x="company_size", y="salary_in_usd_mean", title="Salary Mean vs Company Size", template="seaborn")
fig3.update_layout(bargap=0.3, width=500)
fig3.show() 
# it looks like medium and large companies make 30k-40k more on average than small companies.

In [64]:
# What is the year over year (YoY) salary growth at different levels?
salary_growth_diff_lvls = df.groupby(["work_year", "experience_level"]).agg(pl.col("salary_in_usd").mean().suffix("_mean")).sort("work_year")
salary_growth_diff_lvls

work_year,experience_level,salary_in_usd_mean
i64,str,f64
2020,"""EN""",63648.6
2020,"""MI""",85950.0625
2020,"""SE""",137240.5
2020,"""EX""",202416.5
2021,"""EX""",223752.727273
2021,"""SE""",126596.188406
2021,"""MI""",85490.088889
2021,"""EN""",59101.021277
2022,"""EN""",65423.428571


In [65]:
fig90 = px.bar(data_frame=salary_growth_diff_lvls.to_pandas(), x="experience_level", y="salary_in_usd_mean", facet_col="work_year", template="seaborn")
fig90.show()

In [66]:
salary_growth_diff_lvls_pct_change = salary_growth_diff_lvls.groupby("experience_level").agg(pl.col("work_year"), pl.col("salary_in_usd_mean").pct_change().suffix("_pct_change(%)"))
salary_growth_diff_lvls_pct_change = salary_growth_diff_lvls_pct_change.explode(["salary_in_usd_mean_pct_change(%)", "work_year"]).sort("work_year")
salary_growth_diff_lvls_pct_change = salary_growth_diff_lvls_pct_change.with_columns(pl.col("salary_in_usd_mean_pct_change(%)").apply(lambda x: x*100, return_dtype=pl.Float64))
salary_growth_diff_lvls_pct_change = salary_growth_diff_lvls_pct_change.fill_null(0)
salary_growth_diff_lvls_pct_change

experience_level,work_year,salary_in_usd_mean_pct_change(%)
str,i64,f64
"""EX""",2020,0.0
"""MI""",2020,0.0
"""SE""",2020,0.0
"""EN""",2020,0.0
"""EX""",2021,10.540755
"""MI""",2021,-0.535164
"""SE""",2021,-7.755955
"""EN""",2021,-7.144821
"""EX""",2022,-20.307632


In [67]:
salary_growth_diff_lvls_pct_change = salary_growth_diff_lvls_pct_change.with_columns(pl.when(salary_growth_diff_lvls_pct_change["salary_in_usd_mean_pct_change(%)"]<0).then("red").otherwise("blue").alias("Color"))
salary_growth_diff_lvls_pct_change

experience_level,work_year,salary_in_usd_mean_pct_change(%),Color
str,i64,f64,str
"""EX""",2020,0.0,"""blue"""
"""MI""",2020,0.0,"""blue"""
"""SE""",2020,0.0,"""blue"""
"""EN""",2020,0.0,"""blue"""
"""EX""",2021,10.540755,"""blue"""
"""MI""",2021,-0.535164,"""red"""
"""SE""",2021,-7.755955,"""red"""
"""EN""",2021,-7.144821,"""red"""
"""EX""",2022,-20.307632,"""red"""


In [68]:
salary_growth_diff_lvls_pct_change_fig = px.bar(data_frame=salary_growth_diff_lvls_pct_change.to_pandas(), x="experience_level", y="salary_in_usd_mean_pct_change(%)", facet_col="work_year", text_auto=True, color='Color', template="seaborn")
salary_growth_diff_lvls_pct_change_fig.update_layout(showlegend=False)
salary_growth_diff_lvls_pct_change_fig.update_xaxes(categoryarray=["EN", "MI", "SE", "EX"])
salary_growth_diff_lvls_pct_change_fig.show()

In [69]:
# fig_names = ["Salary Mean", "Salary Mean Growth Percentage Change"]
# fig_dropdown = html.Div([dcc.Dropdown(id="fig-dropdown",
#                              options=[{'label':x, 'value':x} for x in fig_names],
#                              searchable=False,
#                              clearable=False)])

# fig_plot = html.Div(id="fig_plot")

In [129]:
from datetime import datetime
# Create a dashboard to summarize your insights
app = Dash(__name__)

app.layout = html.Div(
    children=[
        html.H1("Challenge 12: Data Science Salaries"),
        html.Span(children=[
            f"Prepared: {datetime.now().date()}",
            html.Br(),
            "Created by ",
            html.A("Ryan Jacobs", href="https://www.linkedin.com/in/ryan-jacobs-08027024a/"),
            html.Br(),
            html.I("Student at University of North Carolina at Charlotte")
        ]),
        html.Ul(children=[
            html.Li(children=[
                html.A("Data in Motion Website", href="https://d-i-motion.com/")
            ]),
            html.Li(children=[
                html.A("GitHub Repo", href="https://github.com/RJUNCC/DataInMotionDSProjects/tree/2302336115ffdab0601095fc2d49efd65b229c2a/Data%20Analysis%20Challenges")
            ]),
            html.Li(children=[
                html.A("LinkedIn", href="https://www.linkedin.com/in/ryan-jacobs-08027024a/")
            ]),
            html.Li(children=[
                html.A("Kaggle", href="https://www.kaggle.com/riainoo")
            ])
        ]),
        # dcc.Dropdown()
        dcc.Tabs([

            # first tab
            dcc.Tab(label='Company Size Plots', children=[

                # first graph
                # html.H4("Fig. 1"),
                dcc.Graph(id="Figure 1", figure=fig3, style={'display':'inline-block', 'width':'100%', 'vertical-align':'top'}),

                # second graph
                # html.H4("Fig. 2"),
                html.Div(children=[
                    dcc.Graph(id="Figure 2", figure=fig2),
                    html.Div(id="desc-for-fig-2", 
                             children="The average salary of company sizes have not changed that much. Medium company sizes have been volatile: ~ -40k change in 2021, ~ +50k change in 2022.",
                             style={'text-align':'center',
                                    'margin':'10px 100px 10px',
                                    'font-family':'monospace',
                                    'font-size':'large'}
                             )],
                         style={'display':'inline-block',
                                'width':'50%',
                                'margin':'5px 0px 0px, 5px', 
                                'vertical-align':'top'}
                        ),

                # third graph
                 html.Div(children=[
                    dcc.Graph(id='Figure 3', figure=fig),
                    html.Div(id="desc-for-fig-3",
                             children="Medium company sizes have increased by a lot between 2020 and 2022, taking the lead over large sized companies in 2022.",
                             style={'text-align':'center',
                                    'margin':'10px 100px 10px',
                                    'font-family':'monospace',
                                    'font-size':'large'}
                            )],
                            style={'display':'inline-block',
                                 'width':'50%',
                                 'margin':'5px 0px 0px, 5px', 
                                 'vertical-align':'top'})
            ]),

            dcc.Tab(label="Experience Level Plots", children=[

                # first graph
                dcc.Graph(id="Figure 4", figure=exp_lvl_highest_hiring_fig, style={'width':'100%', 'display':'inline-block'}),

                # fig_dropdown,
                # fig_plot
                html.Div([
                    dcc.Dropdown(id="dropdown",
                             options=[{'label':"Salary mean differed by year", 'value':'Salary mean differed by year'},
                                      {'label':"Salary growth percent change", 'value': "Salary growth percent change"}],
                                 value="Salary mean differed by year"), 
                    html.Div(dcc.Graph(id="graph")),
                    html.Div(id="desc-for-figs-dropdown",
                             children=["The change in Executive's salary mean decreased by 20%. This is because of the Executives' committee's compensation that was reduced to base salary and board fee.\n", 
                                       html.A("[Source]", href="https://corpgov.law.harvard.edu/2020/08/21/the-pandemic-and-executive-pay/")],
                             style={'text-align':'center',
                                    'margin':'10px 100px 10px',
                                    'font-family':'monospace',
                                    'font-size':'large'})
                             
                             ])

                # fig_plot = html.Div(id="fig_plot"),
                # # second graph
                # dcc.Graph(id='Figure 5', figure=salary_growth_diff_lvls_pct_change_fig, style={'width':'50%', 'display':'inline-block'}),

                # # third graph
                # dcc.Graph(id='Figure 5', figure=fig90, style={'width':'50%', 'display':'inline-block'})
            ]),

            dcc.Tab(label="Employee Demographics", children=[
                html.Div([
                    dcc.Dropdown(id="dropdown1",
                                 options=[{'label':'Job Title Count', 'value':'Job Title Count'},
                                           {'label':'Salary vs Job Title', 'value':'Salary vs Job Title'},
                                           {'label':'Employee Residency', 'value':'Employee Residency'},
                                           {'label': 'Remote Ratio', 'value':'Remote Ratio'},
                                           {'label':'Salary vs Residence vs Company Location', 'value':'Salary vs Residence vs Company Location'},
                                           {'label':'Salary vs Remote Choice vs Residence', 'value':'Salary vs Remote Choice vs Residence'},
                                           {'label':'Salary vs Remote Choice vs Experience', 'value':'Salary vs Remote Choice vs Experience'}],
                                    value="Job Title Count"),
                    html.Div(dcc.Graph(id="graph1")),
                    html.Div(id="desc-for-figs-dropdown1",
                             children=["Data Scientist was the most hired role during the years 2020-2022. USA was where most employees had their residency. The highest paying role of the whole dataset is a Principal Data Engineer: $600k. Most people work remotely."],
                             style={'text-align':'center',
                                    'margin':'10px 100px 10px',
                                    'font-family':'monospace',
                                    'font-size':'large'})
                    
                ])
            ])
        ])
    ]
)

@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    if value == "Salary mean differed by year":
        return fig90
    elif value == "Salary growth percent change":
        return salary_growth_diff_lvls_pct_change_fig

@app.callback(
    Output('graph1', 'figure'),
    [Input(component_id='dropdown1', component_property='value')]
)
def select_graph1(value):
    if value == 'Job Title Count':
        return entry_leveled_fig
    elif value == 'Salary vs Job Title':
        return country_highest_pay_fig
    elif value == "Employee Residency":
        return emp_residence_fig
    elif value == "Remote Ratio":
        return remote_choice_fig
    elif value == "Salary vs Residence vs Company Location":
        return residence_company_location_fig
    elif value == "Salary vs Remote Choice vs Residence":
        return remote_choice_residence_fig
    elif value == "Salary vs Remote Choice vs Experience":
        return remote_choice_experience_fig
    
# def update_output(fig_name):
#     return name_to_figure(fig_name)

# def name_to_figure(fig_name):
#     # figure = go.Figure()
#     if fig_name == "Salary Mean":
#         # figure.add
#         return dcc.Graph(figure=fig90)
#     elif fig_name == "Salary Mean Growth Percentage Change":
#         return dcc.Graph(figure=salary_growth_diff_lvls_pct_change_fig)

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [19/Jun/2023 01:42:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 01:42:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 01:42:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 01:42:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 01:42:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 01:42:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 01:42:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2023 01:42:53] "POST /_dash-update-component HTTP/1.1" 200 -
